In [1]:
!pip install --upgrade transformers>=4.39.2 accelerate>=0.30.0 torch --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.9.0+cu126 requires torch==2.9.0, but you have torch 2.9.1 which is incompatible.
torchvision 0.24.0+cu126 requires torch==2.9.0, but you have torch 2.9.1 which is incompatible.


In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA L4


In [4]:
from sentence_transformers import SentenceTransformer

EMBEDDING_MODEL_NAME = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"

print("Loading embedding model...")
model = SentenceTransformer(
    EMBEDDING_MODEL_NAME,
    trust_remote_code=True,
)
model.max_seq_length = 8192
print("Model loaded.")

Loading embedding model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

Model loaded.


In [18]:
import sqlite3
import json
import os
import numpy as np
from tqdm import tqdm


DB_PATH = "clarification_texts_llama2.db"
TABLE_NAME = "clarifications"
BATCH_SIZE = 32

def get_texts_to_process(db_path: str = DB_PATH):

    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Database file {db_path} not found.")

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{TABLE_NAME}';")
    if not cursor.fetchone():
        conn.close()
        raise Exception(f"Table '{TABLE_NAME}' does not exist in the database.")

    cursor.execute(
        f"""
        SELECT id, concatenated_text
        FROM {TABLE_NAME}
        ORDER BY id ASC
        """
    )
    rows = cursor.fetchall()
    conn.close()

    return rows

def update_embeddings_batch(model, text_ids, texts, db_path: str = DB_PATH):

    if not texts:
        return 0

    valid_indices = []
    ids_to_delete = []

    for i, text in enumerate(texts):
        if not text or not text.strip():
            continue

        if "null" in text:
            print(f"Deleting record containing 'null': {text[:100]}...")
            ids_to_delete.append(text_ids[i])
            continue

        valid_indices.append(i)

    if ids_to_delete:
        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            cursor.executemany(
                f"DELETE FROM {TABLE_NAME} WHERE id = ?",
                [(i,) for i in ids_to_delete]
            )
            conn.commit()
            conn.close()
        except Exception as e:
            print(f"  Error deleting records: {e}")

    if not valid_indices:
        return 0

    valid_text_ids = [text_ids[i] for i in valid_indices]
    valid_texts = [texts[i] for i in valid_indices]

    embeddings_list = []

    try:

        embeddings = model.encode(
            valid_texts,
            batch_size=len(valid_texts),
            show_progress_bar=False,
            convert_to_numpy=True,
            normalize_embeddings=True
        )
        embeddings_list = embeddings

    except Exception as e:
        print(f"  Error computing embeddings batch: {e}")
        return 0

    if len(embeddings_list) == 0:
        return 0


    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    count = 0
    try:
        data_to_update = []
        for text_id, emb in zip(valid_text_ids, embeddings_list):
            emb_json = json.dumps(emb.tolist())
            data_to_update.append((emb_json, text_id))

        cursor.executemany(
            f"UPDATE {TABLE_NAME} SET embedding = ? WHERE id = ?",
            data_to_update
        )
        conn.commit()
        count = len(data_to_update)
    except Exception as e:
        print(f"  Error updating database: {e}")
    finally:
        conn.close()

    return count

def get_statistics(db_path: str = DB_PATH):
    print(f"Debug: Checking database at {os.path.abspath(db_path)}")
    if not os.path.exists(db_path):
        print("Debug: File does not exist.")
        return {"total": 0, "with_embedding": 0, "without_embedding": 0}

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    try:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        # print(f"Debug: Found tables: {tables}")

        cursor.execute(f"SELECT COUNT(*) FROM {TABLE_NAME}")
        total = cursor.fetchone()[0]
        cursor.execute(
            f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE embedding IS NOT NULL"
        )
        with_embedding = cursor.fetchone()[0]
        cursor.execute(
            f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE embedding IS NULL"
        )
        without_embedding = cursor.fetchone()[0]
    except sqlite3.Error as e:
        print(f"Debug: Database error: {e}")
        total = 0
        with_embedding = 0
        without_embedding = 0

    conn.close()

    return {
        "total": total,
        "with_embedding": with_embedding,
        "without_embedding": without_embedding,
    }


print("=" * 80)
print("Computing Embeddings for Database (Overwriting Mode)")
print("=" * 80 + "\n")
if 'model' not in globals():
    print("❌ Error: Model variable 'model' is not defined!")
    print("Please load the model first using the previous cell.")
else:
    print("✓ Model is loaded")

    try:
        if hasattr(model, 'modules') and len(list(model.modules())) > 0:
             transformer = model[0]
             if hasattr(transformer, 'auto_model'):
                 transformer.auto_model.config.use_cache = False
                 print("✓ Disabled use_cache in underlying model config")
    except Exception as e:
        print(f"⚠️ Warning: Could not disable use_cache: {e}")
    try:
        stats = get_statistics(DB_PATH)
        print("Initial Statistics:")
        print(f"  Total texts: {stats['total']}")
        print(f"  With embedding: {stats['with_embedding']}")
        print(f"  Without embedding: {stats['without_embedding']}\n")
        if stats['total'] > 0:
            print("Fetching ALL texts (overwriting existing embeddings)...")
            try:
                rows = get_texts_to_process(DB_PATH)
                print(f"Found {len(rows)} texts to process.")

                text_ids = [r[0] for r in rows]
                texts = [r[1] for r in rows]

                total_updated = 0
                for i in tqdm(range(0, len(texts), BATCH_SIZE), desc="Processing batches"):
                    batch_ids = text_ids[i : i + BATCH_SIZE]
                    batch_texts = texts[i : i + BATCH_SIZE]

                    updated = update_embeddings_batch(model, batch_ids, batch_texts, DB_PATH)
                    total_updated += updated

                print(f"\n✓ Successfully updated {total_updated} embeddings.\n")

                final_stats = get_statistics(DB_PATH)
                print("Final Statistics:")
                print(f"  Total texts: {final_stats['total']}")
                print(f"  With embedding: {final_stats['with_embedding']}")
                print(f"  Without embedding: {final_stats['without_embedding']}\n")

            except Exception as e:
                print(f"\n❌ Error during processing: {e}")
        else:
            print(f"⚠️ No texts found in the database. Please check table '{TABLE_NAME}'.")

    except Exception as e:
        print(f"❌ Error checking database: {e}")

Computing Embeddings for Database (Overwriting Mode)

✓ Model is loaded
✓ Disabled use_cache in underlying model config
Debug: Checking database at /content/clarification_texts_llama2.db
Initial Statistics:
  Total texts: 4968
  With embedding: 4968
  Without embedding: 0

Fetching ALL texts (overwriting existing embeddings)...
Found 4968 texts to process.


Processing batches:   0%|          | 0/156 [00:00<?, ?it/s]

Deleting record containing 'null': Who starred in the Broadway production of Barefoot in the Park? null...
Deleting record containing 'null': The question seems to be asking for the start and end dates of the Manhattan Project. However, the p...
Deleting record containing 'null': When did the Manhattan Project begin and end? null...
Deleting record containing 'null': null null...
Deleting record containing 'null': When did the Manhattan Project begin and end? null...
Deleting record containing 'null': When was the last time the Georgia Bulldogs won a national championship? null...
Deleting record containing 'null': When was the last time the University of Georgia (UGA) won a national championship? null...
Deleting record containing 'null': When did the University of Georgia (UGA) win a national championship? null...
Deleting record containing 'null': When was the last time the University of Georgia (UGA) won a national championship in football? null...


Processing batches:   1%|          | 1/156 [00:00<01:40,  1.54it/s]

Deleting record containing 'null': Voice of the snake in the jungle book null...
Deleting record containing 'null': Voice of the snake in the Jungle Book null...
Deleting record containing 'null': Voice of the snake in the Jungle Book null...


...

Deleting record containing 'null': Who has the most NBA Defensive Player of the Year awards? null...
Deleting record containing 'null': Who has the most NBA Defensive Player of the Year awards? null...
Deleting record containing 'null': Most total goals scored by a team in a Premier League season null...
Deleting record containing 'null': Most total goals scored by a team in a single Premier League season null...
Deleting record containing 'null': Most total goals scored by a player in a Premier League season null...
Deleting record containing 'null': When does Season 7 of "Are You The One?" premiere in 2018. null...
Deleting record containing 'null': When will Season 7 of a specific show be released? null...
Deleting record containing 'null': When does the 2018 movie "Are You the One?" come on (i.e., when is it released)? null...
Deleting record containing 'null': When does the 2018 film "Are You the One?" come on (i.e., when is it released or available to watch)...


Processing batches:  99%|█████████▊| 154/156 [01:19<00:00,  2.09it/s]

Deleting record containing 'null': When was the song "I Don't Want to Miss a Thing" by Aerosmith released? null...
Deleting record containing 'null': When was the song "I Don't Wanna Miss a Thing" by Aerosmith released? null...
Deleting record containing 'null': When was the California Academy of Sciences in San Francisco built? null...
Deleting record containing 'null': When was the California Academy of Sciences building constructed? null...
Deleting record containing 'null': When was the California Academy of Sciences building constructed? null...
Deleting record containing 'null': When was the California Academy of Sciences building project initiated or completed? null...
Deleting record containing 'null': When was the California Academy of Sciences building in San Francisco completed or opened? null...
Deleting record containing 'null': When was the California Academy of Sciences building designed or constructed? null...


Processing batches: 100%|██████████| 156/156 [01:19<00:00,  1.96it/s]

Deleting record containing 'null': Who is the actor that plays Legolas in the Lord of the Rings? null...

✓ Successfully updated 3984 embeddings.

Debug: Checking database at /content/clarification_texts_llama2.db
Final Statistics:
  Total texts: 3984
  With embedding: 3984
  Without embedding: 0



In [21]:
import sqlite3
import pandas as pd

DB_PATH = "clarification_texts_llama2.db"
TABLE_NAME = "clarifications"

def inspect_database(db_path, table_name, limit=25):
    if not os.path.exists(db_path):
        print(f"Database file {db_path} not found.")
        return

    conn = sqlite3.connect(db_path)
    try:

        query = f"SELECT * FROM {table_name} LIMIT {limit}"
        df = pd.read_sql_query(query, conn)

        if 'embedding' in df.columns:
            df['embedding_preview'] = df['embedding'].apply(
                lambda x: (str(x)[:50] + '...') if x else 'NULL'
            )

            df_display = df.drop(columns=['embedding'])
        else:
            df_display = df

        print(f"Displaying first {limit} rows from '{table_name}':")
        display(df_display)

        embedding_count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table_name} WHERE embedding IS NOT NULL", conn)
        print(f"\nTotal rows with embeddings: {embedding_count.iloc[0]['count']}")

    except Exception as e:
        print(f"Error reading database: {e}")
    finally:
        conn.close()

inspect_database(DB_PATH, TABLE_NAME)

Displaying first 25 rows from 'clarifications':


,id,source_id,ambiguous_question,search_result_idx,concatenated_text,embedding_preview
0,1,-3098213414945179817,Who starred in barefoot in the park on broadway?,1,Who were the original Broadway cast members of...,"[0.006943927612155676, -0.009312990121543407, ..."
1,2,-3098213414945179817,Who starred in barefoot in the park on broadway?,2,Who were the lead actors in the original Broad...,"[0.0179608017206192, -0.013800024054944515, 0...."
2,3,-3098213414945179817,Who starred in barefoot in the park on broadway?,3,Who were the original cast members of the Broa...,"[0.01593475416302681, -0.029690392315387726, -..."
3,4,-3098213414945179817,Who starred in barefoot in the park on broadway?,4,Who were the original cast members of the Broa...,"[0.020545851439237595, -0.033293791115283966, ..."
4,5,-3098213414945179817,Who starred in barefoot in the park on broadway?,5,Who were the cast members of the Broadway prod...,"[0.05153382942080498, -0.03288060054183006, -0..."
5,6,-3098213414945179817,Who starred in barefoot in the park on broadway?,6,Who were the original cast members of the Broa...,"[-0.0025356749538332224, -0.01735968329012394,..."
6,7,-3098213414945179817,Who starred in barefoot in the park on broadway?,7,Who were the main cast members of the Broadway...,"[0.021301470696926117, -0.05190042778849602, 0..."
7,8,-3098213414945179817,Who starred in barefoot in the park on broadway?,8,Who were the original cast members of the Broa...,"[0.007323583588004112, -0.0019264030270278454,..."
8,9,-3098213414945179817,Who starred in barefoot in the park on broadway?,9,Who were the main cast members of the Broadway...,"[0.04986383020877838, -0.03663189709186554, 0...."
9,11,-927805218867163489,When did the manhattan project began and end?,1,When did the Trinity test occur? 16 July.,"[-0.02077292464673519, 0.005393927451223135, 0..."



Total rows with embeddings: 3984
